In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB
from tqdm import tqdm
#8 - 125,11 - 126

In [2]:
clusters = json.load(open('./cluster_7.json'))
clusters = list(
map(lambda x: [0]+x, clusters)
)
for cluster in clusters:
    if 8 in cluster:
        cluster.append(125)
    if 11 in cluster:
        cluster.append(126)
solutions = json.load(open('./primary_model_solutions.json'))

In [3]:
def loop(From, l):
    l.append(From)
    if From == 0:
        return l
    to = np.where(x[From] == 1)
    return loop(to[0][0], l)

In [8]:
loops = []
for i,cluster in enumerate(clusters):
    soln = solutions[str(i)]
    x = np.zeros((len(cluster), len(cluster)))
    Vars = soln['Vars']
    for var in Vars:
        if var['VarName'].startswith('x'):
            indices = var['VarName'].split('[')
            indices = indices[1].split(']')[0]
            indices = indices.split(',')
            indices = int(indices[0]), int(indices[1])
            x[indices] = 1
    for zero_to in np.where(x[0]==1)[0]:
        Loop = map(
            lambda idx: cluster[idx],
            loop(zero_to, [0])
        )
        loops.append(list(Loop))

In [12]:
json.dump(loops, open('./loops_7.json', 'w'), indent=4)